In [2]:
import scipy.io
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from keras.preprocessing import image
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix
import cv2
import dlib
from contextlib import contextmanager
import matplotlib.pyplot as plt
from keras.models import load_model

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [ ]:
mat = scipy.io.loadmat('wiki_crop/wiki.mat')

In [ ]:
instances = mat['wiki'][0][0][0].shape[1]
columns = ["dob", "photo_taken", "full_path", "gender", "name", "face_location", "face_score", "second_face_score"]

In [ ]:
df = pd.DataFrame(index = range(0,instances), columns = columns)

for i in mat:
    if i == "wiki":
        current_array = mat[i][0][0]
        for j in range(len(current_array)):
            df[columns[j]] = pd.DataFrame(current_array[j][0])

In [ ]:
def datenum_to_datetime(datenum):
    days = datenum % 1
    hours = days % 1 * 24
    minutes = hours % 1 * 60
    seconds = minutes % 1 * 60
    
    exact_date = datetime.fromordinal(int(datenum)) + timedelta(days=int(days)) + timedelta(hours=int(hours)) + timedelta(minutes=int(minutes)) + timedelta(seconds=round(seconds)) - timedelta(days=366)
    
    
    return exact_date.year

df['date_of_birth'] = df['dob'].apply(datenum_to_datetime)

In [ ]:
df['age'] = df['photo_taken'] - df['date_of_birth']

# Data Cleaning

In [ ]:
#remove pictures does not include face
df = df[df['face_score'] != -np.inf]
 
#some pictures include more than one face, remove them
df = df[df['second_face_score'].isna()]
 
#check threshold
df = df[df['face_score'] >= 3]
 
#some records do not have a gender information
df = df[~df['gender'].isna()]
 
df = df.drop(columns = ['name','face_score','second_face_score','date_of_birth','face_location'])

#some guys seem to be greater than 100. some of these are paintings. remove these old guys
df = df[df['age'] <= 100]
 
#some guys seem to be unborn in the data set
df = df[df['age'] > 0]

# ..

In [ ]:
target_size = (224, 224)
def getImagePixels(image_path):
    img = image.load_img("wiki_crop/%s" % image_path[0], grayscale=False, target_size=target_size)
    x = image.img_to_array(img).reshape(1, -1)[0]
    #x = preprocess_input(x)
    return x
 
df['pixels'] = df['full_path'].apply(getImagePixels)

In [ ]:
target = df['gender'].values
target_classes = keras.utils.to_categorical(target, 2)

In [9]:
#features = df['pixels'].values
features = []

for i in range(0, df.shape[0]):
    features.append(df['pixels'].values[i])

features = np.array(features)
features = features.reshape(features.shape[0], 224, 224, 3)

NameError: name 'df' is not defined

In [ ]:
features.shape

In [ ]:
features /= 255 #normalize in [0, 1]

In [8]:
x_train, x_valid, y_train, y_valid = train_test_split(features, target_classes, test_size=0.10)

NameError: name 'features' is not defined

# Build VGG16 model

In [ ]:
vgg16_model = keras.applications.vgg16.VGG16()

In [ ]:
# As we can see above, the last dense layer has 1000 output nodes, but we only need 2, thus add all other layers except last one.
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)
    
for layer in model.layers:
    layer.trainable = False
    
model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Without training batches
model.fit(x_train, y_train, batch_size=32, epochs=13, validation_data=(x_valid, y_valid), verbose=1)

In [ ]:
model.save('gender_wiki_new.h5')
model.save_weights('gender_wiki_new_weights.h5')

In [ ]:
predictions = model.predict(x_valid)

In [ ]:
pred_list = []; actual_list = []

for i in predictions:
    pred_list.append(np.argmax(i))

for i in y_valid: 
    actual_list.append(np.argmax(i))

confusion_matrix(actual_list, pred_list)